In [47]:
import qutip as qt
import pandas as pd
import numpy as np

import os
import glob

from scipy.optimize import minimize_scalar


In [2]:

# list the folders in ../data
folders = glob.glob("../data/*")

folders = [f for f in folders if "1qb" in f]

# sort the folders by name
folders.sort()

folders


['../data/2025-05-13--16h-14m--2025-05-13--16h-15m_1qb_tomo_sagnac2_40mW_nominally_H',
 '../data/2025-05-13--16h-29m--2025-05-13--16h-30m_1qb_tomo_sagnac2_40mW_B_nominally_H',
 '../data/2025-05-13--17h-01m--2025-05-13--17h-03m_1qb_tomo_sagnac2_40mW_B_nominally_H',
 '../data/2025-05-13--17h-05m--2025-05-13--17h-07m_1qb_tomo_sagnac2_40mW_A_nominally_H',
 '../data/2025-05-14--10h-04m--2025-05-14--10h-06m_1qb_tomo_sagnac2_40mW_A_nominally_V',
 '../data/2025-05-14--10h-55m--2025-05-14--10h-56m_1qb_tomo_sagnac2_40mW_A_nominally_D',
 '../data/2025-05-14--10h-57m--2025-05-14--10h-58m_1qb_tomo_sagnac2_40mW_A_nominally_R',
 '../data/2025-05-14--10h-59m--2025-05-14--11h-00m_1qb_tomo_sagnac2_40mW_A_nominally_A',
 '../data/2025-05-14--11h-00m--2025-05-14--11h-02m_1qb_tomo_sagnac2_40mW_B_nominally_L',
 '../data/2025-05-14--11h-11m--2025-05-14--11h-12m_1qb_tomo_sagnac2_40mW_B_nominally_D',
 '../data/2025-05-14--11h-13m--2025-05-14--11h-14m_1qb_tomo_sagnac2_40mW_B_nominally_R',
 '../data/2025-05-14--1

In [3]:
def extract_reconstructed_state(data, key="reconstructed_state"):

    matrix_11 = data[key][0][0]['real'] + 1j * data[key][0][0]['imag']
    matrix_12 = data[key][0][1]['real'] + 1j * data[key][0][1]['imag']
    matrix_21 = data[key][1][0]['real'] + 1j * data[key][1][0]['imag']
    matrix_22 = data[key][1][1]['real'] + 1j * data[key][1][1]['imag']

    matrix = [[matrix_11, matrix_12],
                [matrix_21, matrix_22]]
    return qt.Qobj(matrix, dims=[[2], [2]])

In [4]:
# for each folder, load the data in .../R/tomography_resutls.json and .../T/tomography_results.json

reconstructed_states = []

for folder in folders:
    last_part_of_folder = folder.split("/")[-1]

    # folder names like 2025-05-13--16h-29m--2025-05-13--16h-30m_1qb_tomo_sagnac2_40mW_B_nominally_H
    # extract the parameters from the folder name

    params = last_part_of_folder.split("_")
    nominal_state_name = params[-1]
    launcher = params[-3]
    laser_power = params[-4]

    R_file = os.path.join(folder, "R", "tomography_results.json")
    T_file = os.path.join(folder, "T", "tomography_results.json")

    if not os.path.exists(R_file) or not os.path.exists(T_file):
        print(f"Skipping {folder} because one of the files does not exist")
        continue

    R_data = pd.read_json(R_file)
    T_data = pd.read_json(T_file)

    rho_R = extract_reconstructed_state(R_data)
    rho_T = extract_reconstructed_state(T_data)
    
    target_state = extract_reconstructed_state(T_data, key="target_state")

    reconstructed_states.append({
        "folder": folder,
        "nominal_state_name": nominal_state_name,
        "launcher": launcher,
        "laser_power": laser_power,
        "reconstructed_state_T": rho_T,
        "reconstructed_state_R": rho_R,
        "target_state": target_state
    })

reconstructed_states[-1]['reconstructed_state_R']



Skipping ../data/2025-05-13--16h-14m--2025-05-13--16h-15m_1qb_tomo_sagnac2_40mW_nominally_H because one of the files does not exist


Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.49766767+0.j         0.32104273+0.37656206j]
 [0.32104273-0.37656206j 0.50233233+0.j        ]]

In [37]:
reconstructed_state_T = reconstructed_states[-1]['reconstructed_state_T']
reconstructed_state_T

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.5146096 +0.j         -0.09375157-0.48395731j]
 [-0.09375157+0.48395731j  0.4853904 +0.j        ]]

In [6]:
reconstructed_states[-1]['nominal_state_name']

'L'

In [10]:
reconstructed_states[-1]['launcher']

'B'

In [11]:
target_state = reconstructed_states[-1]['target_state']

We need to find the unitary rotation of the bloch sphere in the X-Y plane that rotates the target state onto the reconstructed state

In [28]:
# rotation around the x axis of the Bloch sphere
def rotation_matrix_x(theta):

    phase = np.exp(1j * theta / 2)
    m = np.array([[1, 0],
                     [0, phase]])
    return qt.Qobj(m, dims=[[2], [2]])

def rotate_target_state(theta, target_state):
    """
    Rotate the target state around the x axis of the Bloch sphere by theta
    """
    rotation = rotation_matrix_x(theta)
    return rotation * target_state * rotation.dag()

In [32]:
rotated_target_state = rotate_target_state(0.1, target_state)
rotated_target_state

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.5       +0.00000000e+00j -0.02498958-4.99375130e-01j]
 [-0.02498958+4.99375130e-01j  0.5       -2.10657063e-18j]]

In [86]:

def trace_norm_distance(rho1, rho2):
    """
    Compute the trace norm distance between two density matrices
    """
    return (rho1 - rho2).norm('tr')

def find_best_rotation_angle(target_state, reconstructed_state):
    """
    Use scipy to find the best rotation angle that minimizes the trace norm distance
    """
    def objective_function(theta):
        rotated_state = rotate_target_state(theta, target_state)
        return trace_norm_distance(rotated_state, reconstructed_state)
    result = minimize_scalar(objective_function, bounds=(-2*np.pi, 2*np.pi), method='bounded')
    best_angle = result.x
    best_distance = result.fun
    return best_angle, best_distance


In [87]:
best_angle, best_distance = find_best_rotation_angle(target_state, reconstructed_state_T)
best_angle, best_distance

(np.float64(0.3826960619811966), np.float64(0.03243954743465447))

In [72]:
# Show the rotated target state
rotated_target_state = rotate_target_state(best_angle, target_state)
rotated_target_state

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.5       +0.00000000e+00j -0.09509125-4.90874378e-01j]
 [-0.09509125+4.90874378e-01j  0.5       +1.15630190e-18j]]

In [73]:
reconstructed_state_T

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.5146096 +0.j         -0.09375157-0.48395731j]
 [-0.09375157+0.48395731j  0.4853904 +0.j        ]]

In [79]:
# for all the reconstructed states, find the best rotation angle and distance
best_angles = []
best_distances = []
tomos = []
for tomo in ['R', 'T']:
    for state in reconstructed_states:
        target_state = state['target_state']
        reconstructed_state = state[f'reconstructed_state_{tomo}']
        # find the best rotation angle and distance
        best_angle, best_distance = find_best_rotation_angle(target_state, reconstructed_state)
        best_angles.append(best_angle * 180 / np.pi % 360)
        best_distances.append(best_distance)
        tomos.append(tomo)

# create a dataframe with the results  
df = pd.DataFrame({
    "folder": [state['folder'] for state in reconstructed_states]*2,
    "tomo": tomos,
    "best_angle": best_angles,
    "best_distance": best_distances,
    "nominal_state_name": [state['nominal_state_name'] for state in reconstructed_states]*2,
    "launcher": [state['launcher'] for state in reconstructed_states]*2
    })
df.to_csv("best_angles.csv", index=False)

In [80]:
reconstructed_states[-4]['reconstructed_state_T']

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.51690695+0.j         -0.48515256+0.09100134j]
 [-0.48515256-0.09100134j  0.48309305+0.j        ]]